<a href="https://colab.research.google.com/github/BorisLoveDev/agents-of-uniform-difficulty/blob/main/save_combinded_diffiiculty_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import os
import random
np.random.seed(42)
random.seed(42)

In [4]:
from google.colab import drive
# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Загрузка и нормализация данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

num_classes = 10

# Добавление измерения канала к данным
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Применение one-hot encoding
y_test_ohe = to_categorical(y_test, num_classes)
y_train_ohe = to_categorical(y_train, num_classes)
y_train_ohe.shape

11490434/11490434 [==============================] - 1s 0us/step


(60000, 10)

In [10]:
# Загрузка данных для уровня сложности 0
x_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_0.npy')
y_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_0.npy')

# Загрузка данных для уровня сложности 6
x_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_6.npy')
y_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_6.npy')


In [8]:
# Функция для деформации изображения, как определено ранее
def deform_image_optimized(image, A, B, M=28, NP=5):
    C = M / (NP + 1.0)
    XN, YN = np.zeros(M), np.zeros(M)
    DX, DY = np.linspace(0, M-1, M), np.linspace(0, M-1, M)

    for j in range(NP):
        TXN = (j + 0.5 - np.random.random()) * C
        TYN = (j + 0.5 - np.random.random()) * C
        TDX = (j + 0.5 - np.random.random()) * C
        TDY = (j + 0.5 - np.random.random()) * C
        AXN = B * (1.0 - 2.0 * np.random.random())
        AYN = B * (1.0 - 2.0 * np.random.random())
        ADX = A * (1.0 - 2.0 * np.random.random())
        ADY = A * (1.0 - 2.0 * np.random.random())
        PXN = (0.1 + 0.9 * np.random.random()) * C
        PYN = (0.1 + 0.9 * np.random.random()) * C
        PDX = (0.1 + 0.9 * np.random.random()) * C
        PDY = (0.1 + 0.9 * np.random.random()) * C

        DX += ADX * np.exp(-((DX - TDX) / PDX)**2)
        DY += ADY * np.exp(-((DY - TDY) / PDY)**2)
        XN += AXN * np.exp(-((DX - TXN) / PXN)**2)
        YN += AYN * np.exp(-((DY - TYN) / PYN)**2)

    deformed_image = np.zeros((M, M))
    for j in range(M):
        for i in range(M):
            x_index = int(DX[i] + XN[j])
            y_index = int(DY[j] + YN[i])
            if 0 <= x_index < M and 0 <= y_index < M:
                deformed_image[j, i] = image[y_index, x_index] if y_index < image.shape[0] and x_index < image.shape[1] else 0

    return deformed_image

In [9]:
deform_params = {
    "1": (0.2, 1.0),
    "2": (0.4, 3.0),
    "3": (1, 5.0),
    "4": (3.5, 9.5),
    "5": (5, 15.0),
}

In [17]:
# Взятие по 1500 изображений из уровней сложности 0 и 6
x_data_level_0_subset = x_data_level_0[:1500]
y_data_level_0_subset = y_data_level_0[:1500]
x_data_level_6_subset = x_data_level_6[:1500]
y_data_level_6_subset = y_data_level_6[:1500]

# Убедимся, что все массивы имеют одинаковое количество измерений
x_data_level_0_subset = np.expand_dims(x_data_level_0_subset, axis=-1) if x_data_level_0_subset.ndim == 3 else x_data_level_0_subset
x_data_level_6_subset = np.expand_dims(x_data_level_6_subset, axis=-1) if x_data_level_6_subset.ndim == 3 else x_data_level_6_subset

# Инициализация списка для объединения с проверкой размерности
x_combined = [x_data_level_0_subset]
y_combined = [y_data_level_0_subset]

# Генерация деформированных изображений для уровней сложности с 1 по 5 и добавление их в список
for level, params in deform_params.items():
    A, B = params
    start_index = 1500 * int(level)  # Исправленный начальный индекс для x_test
    end_index = start_index + 1500
    x_subset = x_test[start_index:end_index]
    y_subset = y_test[start_index:end_index]

    # Деформация изображений
    x_deformed = np.array([deform_image_optimized(img.squeeze(), A, B) for img in x_subset])
    x_deformed = np.expand_dims(x_deformed, axis=-1)  # Проверка не требуется, всегда добавляем измерение канала

    x_combined.append(x_deformed)
    y_combined.append(y_subset)

# Добавление изображений уровня сложности 6 в конец
x_combined.append(x_data_level_6_subset)
y_combined.append(y_data_level_6_subset)

# Преобразование списков в массивы Numpy и их объединение
x_combined = np.concatenate(x_combined, axis=0)
y_combined = np.concatenate(y_combined, axis=0)

# Преобразование меток в one-hot encoding
#y_combined_ohe = to_categorical(y_combined, num_classes)

print(f"Форма итогового набора данных: {x_combined.shape}")
print(f"Форма итоговых меток: {y_combined.shape}")


Форма итогового набора данных: (10500, 28, 28, 1)
Форма итоговых меток: (10500,)


In [18]:
# Путь к директории на Google Drive
save_dir = '/content/drive/MyDrive/all_difficulty_data_mnist'

# Создание директории, если она еще не существует
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Пути для сохранения файлов
x_save_path = os.path.join(save_dir, 'x_combined_difficulty.npy')
y_save_path = os.path.join(save_dir, 'y_combined_difficulty.npy')
#y_ohe_save_path = os.path.join(save_dir, 'y_combined_difficulty_ohe.npy')

# Сохранение наборов данных
np.save(x_save_path, x_combined)
np.save(y_save_path, y_combined)
#np.save(y_ohe_save_path, y_combined_ohe)

print(f'Наборы данных сохранены в {save_dir}')


Наборы данных сохранены в /content/drive/MyDrive/all_difficulty_data_mnist
